# City Schools Analysis

 - OBSERVED TREND 1 : Students at Charter Schools have better overall passing rate as compared to District Schools.

 - OBSERVED TREND 2 : Schools with large number of students have poorer overall passing rate.

 - OBSERVED TREND 3 : Reading scores are generally better than Math scores for most of the students.

In [1]:
import pandas as pd
import numpy as np
import os

schoolDataFile = os.path.join("raw_data","schools_complete.csv")
studentsDataFile = os.path.join("raw_data","students_complete.csv")

schools = pd.read_csv(schoolDataFile)
students = pd.read_csv(studentsDataFile)

## District Summary Analysis

In [2]:
is_district = schools["type"] == "District"
students_dist = students.loc[students["school"].isin(schools[is_district]['name'].tolist())]
df_ds = pd.DataFrame(
    {"Total Schools": [schools[is_district]['name'].count()],
     "Total Students": [schools[is_district]['size'].sum()],
     "Total Budget": [schools[is_district]['budget'].sum()],
     "Average Math Score": [students.loc[students["school"].isin(schools[is_district]['name'].tolist())]["math_score"].mean()],
     "Average Reading Score": [students.loc[students["school"].isin(schools[is_district]['name'].tolist())]["reading_score"].mean()],
     "% Passing Math": [students_dist.loc[students_dist["math_score"] >= 70]["name"].count() / students_dist["name"].count() * 100],
     "% Passing Reading": [students_dist.loc[students_dist["reading_score"] >= 70]["name"].count() / students_dist["name"].count() * 100],
     "% Overall Passing Rate": [(students_dist.loc[students_dist["math_score"] >= 70]["name"].count() / students_dist["name"].count() + students_dist.loc[students_dist["reading_score"] >= 70]["name"].count() / students_dist["name"].count()) / 2 * 100],
    }
)
df_ds['Total Budget'] = df_ds['Total Budget'].map('${:,.2f}'.format)
df_ds = df_ds[["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
df_ds

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,7,26976,"$17,347,923.00",76.987026,80.962485,66.518387,80.905249,73.711818


## School Summary Analysis

In [3]:
df_ssa1 = schools
df_ssa1 = df_ssa1.set_index(['name'])
df_ssa1 = df_ssa1[['type','size','budget']]
df_ssa1['Per Student Budget'] = df_ssa1['budget'] / df_ssa1['size']
df_ssa2 = students.groupby('school').mean()
df_ssa2 = df_ssa2[['math_score','reading_score']]
df_ssa2.index.name = ''
df_ssa1 = df_ssa1.join(df_ssa2)
df_ssa1.columns = ['School Type', 'Total Students', 'Total School Budget','Per Student Budget','Average Math Score', 'Average Reading Score']
df_pmr = pd.DataFrame({'Students Passed Math' : students.loc[(students['math_score'] >= 70)].groupby('school')['math_score'].count(),
                        'Students Passed Reading' : students.loc[(students['reading_score'] >= 70)].groupby('school')['reading_score'].count()})
df_ssa1 = df_ssa1.join(df_pmr)
df_ssa1['% Passing Math'] = df_ssa1['Students Passed Math'] / df_ssa1['Total Students'] * 100
df_ssa1['% Passing Reading'] = df_ssa1['Students Passed Reading'] / df_ssa1['Total Students'] * 100
df_ssa1['% Overall Passing Rate'] = (df_ssa1['% Passing Reading'] + df_ssa1['% Passing Math']) / 2
df_ssa = df_ssa1
df_ssa.index.name = ''
df_ssa = df_ssa[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
df_ss = df_ssa.copy()
df_ss['Per Student Budget'] = df_ssa['Per Student Budget'].map('${:,.2f}'.format) 
df_ss['Total School Budget'] = df_ssa['Total School Budget'].map('${:,.2f}'.format)
df_ss

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391


## Top Performing Schools(By Passsing Rate)

In [4]:
df_ss.sort_values('% Overall Passing Rate', ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools(By Passsing Rate)

In [5]:
df_ss.sort_values('% Overall Passing Rate', ascending=True)[:5]

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade Analysis

In [6]:
msbg = students.groupby(['school','grade'], as_index=False).mean()
msbg = msbg.pivot(index='school',columns='grade',values='math_score')
msbg.index.name = ''
msbg.columns.name = ''
msbg = msbg[['9th','10th','11th','12th']]
msbg

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Scores by Grade Analysis

In [7]:
rsbg = students.groupby(['school','grade'], as_index=False).mean()
rsbg = rsbg.pivot(index='school',columns='grade',values='reading_score')
rsbg.index.name = ''
rsbg.columns.name = ''
rsbg = rsbg[['9th','10th','11th','12th']]
rsbg

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending Analysis

In [8]:
budRg_bin = [0,584,614,644,674]
budRg_GrpNm = ['<$585','$585-615','$615-645','$645-675']
sbssp = df_ssa.copy()
sbssp['BudgetRange'] = pd.cut(sbssp['Per Student Budget'], budRg_bin, labels=budRg_GrpNm)
sbssp = sbssp.groupby('BudgetRange').mean()
sbssp = sbssp[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
sbssp = sbssp.reindex(index=budRg_GrpNm)
sbssp.index.name = 'Spending Ranges (Per Student)'
sbssp

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size Analysis

In [9]:
sizeRg_bin = [0,1000,2000,5000]
sizeRg_GrpNm = ['Small(<1000)','Medium(1000-2000)','Large(2000-5000)']
sbssz = df_ssa.copy()
sbssz['SizeRange'] = pd.cut(sbssz['Total Students'], sizeRg_bin, labels=sizeRg_GrpNm)
sbssz = sbssz.groupby('SizeRange').mean()
sbssz = sbssz[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
sbssz = sbssz.reindex(index=sizeRg_GrpNm)
sbssz.index.name = 'School Size'
sbssz

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type Analysis

In [10]:
sbst = df_ssa
sbst = sbst.groupby('School Type').mean()
sbst = sbst[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
sbst

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
